In [37]:
import os
import sys
from datetime import datetime, timedelta
import logging
import pickle
import time
from dataclasses import dataclass, field
from enum import Enum
from typing import Any, Dict, List, Optional, Set, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import unicodedata
from dotenv import load_dotenv

# Add project root to sys.path
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Verify sys.path
# print("Current sys.path:", sys.path)

from src.data_processing.nst_scraper import nst_on_ice_scraper, nst_team_on_ice_scraper
from src.db.nhl_db_utils import update_player_db, check_last_update, append_player_ids, get_player_full_name
from src.db.the_odds_db_utils import get_team_moneyline_odds
from src.data_processing.team_utils import get_most_recent_game_id, get_fullname_by_tricode
from src.data_processing.game_utils import get_game_boxscore, display_boxscore
from src.data_processing.pbp_utils import get_matchup_games

from src.entities.lineup import Lineup, Player, Position

from dotenv import load_dotenv

pd.set_option('display.max_columns', None)
np.set_printoptions(legacy='1.25')

In [38]:
today_datetime= datetime.now()
yesterday_datetime = today_datetime - timedelta(days=1, hours=6) # UTC offset
yesterday = yesterday_datetime.strftime('%Y-%m-%d')
yesterday

'2025-01-16'

In [39]:
# Load environment variables from .env file
load_dotenv()

db_prefix = 'NHL_DB_'

In [40]:
# Check the last update time of the players database
last_update = check_last_update(db_prefix)

INFO:src.db.base_utils:Database connection established.
INFO:src.db.nhl_db_utils:Last database update was on: 2025-01-15
INFO:src.db.base_utils:Database connection closed.


In [41]:
# Convert last_update to datetime
last_update_dt = datetime.strptime(last_update, '%Y-%m-%d')
today_dt = datetime.strptime(today_datetime.strftime('%Y-%m-%d'), '%Y-%m-%d')
yesterday_dt = datetime.strptime(yesterday, '%Y-%m-%d')

# Only update if last update was before yesterday
if last_update_dt not in [today_dt, yesterday_dt]:
    # Update the player database from last update to yesterday
    update_player_db(last_update, yesterday, db_prefix, skip_existing=True)
else:
    print(f"No need to update the player database. Last update was on: {last_update}")


INFO:src.data_processing.utils:Making GET request to: https://api-web.nhle.com/v1/gamecenter/2024020705/boxscore
INFO:src.db.nhl_db_utils:Successfully fetched boxscore for game 2024020705
INFO:src.data_processing.utils:Making GET request to: https://api-web.nhle.com/v1/gamecenter/2024020706/boxscore
INFO:src.db.nhl_db_utils:Successfully fetched boxscore for game 2024020706
INFO:src.data_processing.utils:Making GET request to: https://api-web.nhle.com/v1/gamecenter/2024020707/boxscore
INFO:src.db.nhl_db_utils:Successfully fetched boxscore for game 2024020707
INFO:src.data_processing.utils:Making GET request to: https://api-web.nhle.com/v1/gamecenter/2024020708/boxscore
INFO:src.db.nhl_db_utils:Successfully fetched boxscore for game 2024020708
INFO:src.data_processing.utils:Making GET request to: https://api-web.nhle.com/v1/gamecenter/2024020709/boxscore
INFO:src.db.nhl_db_utils:Successfully fetched boxscore for game 2024020709
INFO:src.data_processing.utils:Making GET request to: https:

In [42]:
def get_skater_stats(lineup: Lineup, player_stats_df: pd.DataFrame, filter: Optional[str] = None) -> pd.DataFrame:
    """
    Gets stats for players in the lineup, maintaining lineup order.
    
    Args:
        lineup (Lineup): The lineup containing players
        player_stats_df (pd.DataFrame): DataFrame with player statistics
        filter (str, optional): Type of filter to apply on the stats.
        Defaults to None, which keeps all stats. If 'shots', only shot-related statistics are kept.
        
    Returns:
        pd.DataFrame: Player statistics ordered according to lineup positions
    """
    # Apply filter if specified
    if filter == 'shots':
        # Define shot-related columns to keep
        shot_columns = ['player', 'team', 'position', 'gp', 'toi', 'toi/gp','shots/60', 'icf/60', 'iff/60'] 
        player_stats_df = player_stats_df[shot_columns]
    
    # Create ordered list of players (forwards then defense)
    players = []
    # Add forwards in order
    players.extend([p for p in lineup.forwards if p])
    # Add defense in order 
    players.extend([p for p in lineup.defense if p])
    
    # Create ordered list of player names
    player_names = [player.name for player in players]
    
    # Filter stats and reorder to match lineup order
    stats_df = player_stats_df[player_stats_df['player'].isin(player_names)]
    
    # Create ordering dictionary mapping names to their position in lineup
    name_to_position = {name: idx for idx, name in enumerate(player_names)}
    
    # Sort stats DataFrame based on lineup order and reset index
    return stats_df.assign(
        lineup_order=stats_df['player'].map(name_to_position)
    ).sort_values('lineup_order').drop('lineup_order', axis=1).reset_index(drop=True)

# lineup_player_stats = get_skater_stats(my_lineup, player_stats_df)

In [43]:
def get_goalie_stats(lineup: Lineup, goalie_stats_df: pd.DataFrame) -> pd.DataFrame:
    """
    Gets stats for goalies in the lineup, maintaining lineup order.
    
    Args:
        lineup (Lineup): The lineup containing goalies
        goalie_stats_df (pd.DataFrame): DataFrame with goalie statistics
        
    Returns:
        pd.DataFrame: Goalie statistics ordered according to lineup positions
    """
    # Create ordered list of goalies
    goalies = [goalie for goalie in lineup.goalies if goalie]
    goalie_names = [goalie.name for goalie in goalies]
    
    # Filter stats and reorder to match lineup order
    stats_df = goalie_stats_df[goalie_stats_df['player'].isin(goalie_names)]
    
    # Create ordering dictionary mapping names to their position in lineup
    name_to_position = {name: idx for idx, name in enumerate(goalie_names)}
    
    # Sort stats DataFrame based on lineup order and reset index
    return stats_df.assign(
        lineup_order=stats_df['player'].map(name_to_position)
    ).sort_values('lineup_order').drop('lineup_order', axis=1).reset_index(drop=True)

In [44]:
def extract_team_lineup(team: str, reference_date: Optional[str] = None) -> Lineup:
    """
    Extracts the most recent lineup for the specified team based on the latest game data.

    This function performs the following steps:
        1. Determines the reference date (defaults to yesterday if not provided).
        2. Retrieves the most recent game ID for the team using `get_most_recent_game_id`.
        3. Fetches the game boxscore data using `get_game_boxscore`.
        4. Processes the boxscore to obtain skaters and goalies using `display_boxscore`.
        5. Constructs and returns a `Lineup` object populated with the team's players.

    Args:
        team (str): The three-letter team code (e.g., 'TOR').
        reference_date (Optional[str]): The reference date in 'YYYY-MM-DD' format. Defaults to yesterday's date.

    Returns:
        Lineup: A `Lineup` object containing the team's players from the most recent game.

    Raises:
        ValueError: If no recent game is found for the team or if the team is not part of the retrieved game.
    """
    # Step 1: Determine the reference date
    if reference_date is None:
        today_datetime = datetime.now()
        yesterday_datetime = today_datetime - timedelta(days=1, hours=6)  # Adjust for UTC offset if necessary
        reference_date = yesterday_datetime.strftime('%Y-%m-%d')

    # Step 2: Retrieve the most recent game ID for the team
    game_id = get_most_recent_game_id(team, reference_date)
    if game_id is None:
        raise ValueError(f"No recent game found for team '{team}' before {reference_date}.")

    # Print the game_id
    print(f"Game ID: {game_id}")

    # Step 3: Fetch the game boxscore data
    game_data = get_game_boxscore(game_id, clean=False)

    # Step 4: Process the boxscore to obtain skaters and goalies
    away_skaters, away_goalies, home_skaters, home_goalies = display_boxscore(game_data)

    # Extract team abbrevs to determine if the team is home or away
    away_team_code = game_data.get('awayTeam', {}).get('abbrev')
    home_team_code = game_data.get('homeTeam', {}).get('abbrev')

    if not away_team_code or not home_team_code:
        raise ValueError("Team abbreviations not found in game data.")

    if team.upper() == away_team_code.upper():
        team_side = 'Away'
        skaters = away_skaters
        goalies = away_goalies
    elif team.upper() == home_team_code.upper():
        team_side = 'Home'
        skaters = home_skaters
        goalies = home_goalies
    else:
        raise ValueError(f"Team '{team}' not found in game ID {game_id}.")

    # Step 5: Construct the Lineup object
    lineup = Lineup(name=f"{team.upper()} Lineup from Game {game_id}")

    # Add Skaters to the Lineup
    for _, skater in skaters.iterrows():
        try:
            position_enum = Position(skater['position'])  # Convert to Position Enum
        except ValueError:
            print(f"Invalid position '{skater['position']}' for player '{skater['name']}'. Skipping.")
            continue

        player = Player(
            player_id=skater['playerId'],
            name=get_player_full_name(skater['playerId'], db_prefix, suppress_log=True),
            team=team.upper(),
            position=position_enum
        )

        # Add player to the appropriate category in the lineup
        if player.position.category == 'F':
            try:
                empty_slot = next(i for i, p in enumerate(lineup.forwards) if p is None)
                lineup.add_forward(player, empty_slot)
            except StopIteration:
                print(f"No available forward slot to add player '{player.name}'.")
        elif player.position.category == 'D':
            try:
                empty_slot = next(i for i, p in enumerate(lineup.defense) if p is None)
                lineup.add_defense(player, empty_slot)
            except StopIteration:
                print(f"No available defense slot to add player '{player.name}'.")
        else:
            print(f"Player '{player.name}' has an unrecognized category '{player.position.category}'. Skipping.")

    # Add Goalies to the Lineup
    for _, goalie in goalies.iterrows():
        player = Player(
            player_id=goalie['playerId'],
            name=get_player_full_name(goalie['playerId'], db_prefix, suppress_log=True),
            team=team.upper(),
            position=Position.G
        )
        try:
            empty_slot = next(i for i, p in enumerate(lineup.goalies) if p is None)
            lineup.set_goalie(player, empty_slot)
        except StopIteration:
            print(f"No available goalie slot to add player '{player.name}'.")

    return lineup
# chicago_lineup = extract_team_lineup('CHI', '2024-11-22')

In [45]:
def calculate_min_percentage(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates the average time on ice per game as a percentage of total game time (60 minutes).
    
    Args:
        df (pd.DataFrame): DataFrame containing 'toi' and 'gp' columns
        
    Returns:
        pd.DataFrame: Original DataFrame with new 'min%' column added
    """
    df_copy = df.copy()
    df_copy['min%'] = (df_copy['toi'] / df_copy['gp'] / 300 * 100).round(2)
    return df_copy

# # Apply the function to lineup_player_stats
# lineup_player_stats = calculate_min_percentage(lineup_player_stats)
# lineup_player_stats

def sum_min_percentage(df: pd.DataFrame) -> float:
    """
    Calculates the sum of the 'min%' column in the given DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame containing the 'min%' column.
        
    Returns:
        float: The total sum of the 'min%' values.
        
    Raises:
        KeyError: If the 'min%' column is not present in the DataFrame.
    """
    if 'min%' not in df.columns:
        raise KeyError("The DataFrame does not contain a 'min%' column.")
    
    total_min_percentage = df['min%'].sum()
    return total_min_percentage

# Example usage:
# total_min_percentage = sum_min_percentage(lineup_player_stats)
# print(f"Total min%: {total_min_percentage}")

def calculate_adj_min(df: pd.DataFrame, total_min_percentage: float) -> None:
    """
    Calculates the adjusted minimum (adj_min) for each player based on their min% and the total min%.
    
    The formula used is:
        adj_min = (min% / total_min_percentage) * 300
    
    Args:
        df (pd.DataFrame): DataFrame containing the 'min%' column.
        total_min_percentage (float): The total sum of the 'min%' column.
        
    Raises:
        KeyError: If the 'min%' column is not present in the DataFrame.
        ValueError: If total_min_percentage is not a positive number.
    """
    if 'min%' not in df.columns:
        raise KeyError("The DataFrame does not contain a 'min%' column.")
    
    if total_min_percentage <= 0:
        raise ValueError("total_min_percentage must be a positive number.")
    
    # Calculate and append the 'adj_min' column
    df['adj_min'] = ((df['min%'] / total_min_percentage) * 300).round(2)

# # Example usage:
# calculate_adj_min(lineup_player_stats, total_min_percentage)
# print(lineup_player_stats[['player', 'min%', 'adj_min']])

In [46]:
# Define file paths to load the model and transformer
model_filepath = '../models/polynomial_model_degree_1.pkl'
poly_filepath = '../models/polynomial_features_degree_1.pkl'

# Load the regression model
with open(model_filepath, 'rb') as model_file:
    loaded_model = pickle.load(model_file)
print(f"Model loaded from {model_filepath}")

# Load the PolynomialFeatures transformer
with open(poly_filepath, 'rb') as poly_file:
    loaded_poly = pickle.load(poly_file)
print(f"PolynomialFeatures transformer loaded from {poly_filepath}")

# Function to make predictions using the loaded model and transformer
def predict_gpm(new_ixg60_value, model, poly, x_col='ixg60'):
    """
    Predicts 'gpm' using the loaded model and polynomial transformer.
    
    Parameters:
        new_ixg60_value (float): The new ixg60 value for prediction.
        model (RegressionResults): The loaded regression model.
        poly (PolynomialFeatures): The loaded polynomial features transformer.
        x_col (str): The name of the independent variable column. Defaults to 'ixg60'.
        
    Returns:
        predicted_gpm (float): The predicted gpm value.
    """
    # Prepare the input data
    X_new = np.array([[new_ixg60_value]])
    X_new_poly = poly.transform(X_new)
    X_new_poly_const = sm.add_constant(X_new_poly, has_constant='add')
    
    # Create DataFrame with appropriate column names
    feature_names = ['const'] + poly.get_feature_names_out([x_col]).tolist()
    new_data = pd.DataFrame(X_new_poly_const, columns=feature_names)
    
    # Predict
    predicted_gpm = model.predict(new_data)
    return predicted_gpm.iloc[0]

# Example: Predicting 'gpm' for a new ixg60 value
# new_ixg60_value = 50
# predicted_gpm = predict_gpm(new_ixg60_value, loaded_model, loaded_poly)
# print(f"Predicted GPM for ixg60={new_ixg60_value}: {predicted_gpm:.4f}")

Model loaded from ../models/polynomial_model_degree_1.pkl
PolynomialFeatures transformer loaded from ../models/polynomial_features_degree_1.pkl


In [47]:
# Function to predict GPM for each player and add it to the DataFrame
def add_gpm_to_lineup(lineup_df, model, poly):
    """
    Adds a 'gpm' column to the lineup_player_stats DataFrame using the predict_gpm function.
    
    Args:
        lineup_df (pd.DataFrame): DataFrame containing 'ixg/60' column.
        model: Loaded regression model.
        poly: Loaded PolynomialFeatures transformer.
        
    Returns:
        pd.DataFrame: Updated DataFrame with 'gpm' column added.
    """
    # Define a helper function to handle potential missing or invalid values
    def safe_predict(ixg_60):
        if pd.isna(ixg_60):
            return np.nan
        try:
            return predict_gpm(ixg_60, model, poly)
        except Exception as e:
            print(f"Error predicting GPM for ixg_60={ixg_60}: {e}")
            return np.nan
    
    # Apply the predict_gpm function to each 'ixg/60' value
    lineup_df['gpm'] = lineup_df['ixg/60'].apply(safe_predict)
    return lineup_df

# # Apply the function to add 'gpm' to your DataFrame
# lineup_player_stats = add_gpm_to_lineup(lineup_player_stats, loaded_model, loaded_poly)

# # Display the updated DataFrame with 'gpm'
# print(lineup_player_stats[['player', 'ixg/60', 'gpm']])

In [48]:
def calculate_x_goals(lineup_stats_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculates 'x_goals' by multiplying 'gpm' with 'adj_min' for each player.
    
    Args:
        lineup_stats_df (pd.DataFrame): DataFrame containing 'gpm' and 'adj_min' columns.
        
    Returns:
        pd.DataFrame: DataFrame with the new 'x_goals' column added.
        
    Raises:
        KeyError: If 'gpm' or 'adj_min' columns are not present in the DataFrame.
        TypeError: If 'gpm' or 'adj_min' contain non-numeric data.
    """
    # Check if required columns exist
    required_columns = {'gpm', 'adj_min'}
    missing_columns = required_columns - set(lineup_stats_df.columns)
    if missing_columns:
        raise KeyError(f"The DataFrame is missing the following required columns: {', '.join(missing_columns)}")
    
    # Check if 'gpm' and 'adj_min' are numeric
    if not pd.api.types.is_numeric_dtype(lineup_stats_df['gpm']):
        raise TypeError("'gpm' column must be numeric.")
    if not pd.api.types.is_numeric_dtype(lineup_stats_df['adj_min']):
        raise TypeError("'adj_min' column must be numeric.")
    
    # Calculate 'x_goals'
    lineup_stats_df = lineup_stats_df.copy()
    lineup_stats_df['x_goals'] = lineup_stats_df['gpm'] * lineup_stats_df['adj_min']
    
    return lineup_stats_df

In [49]:
def calculate_league_avg_xg_against_per_60(goalie_stats_df: pd.DataFrame) -> float:
    """
    Calculate the league average expected goals against (xg_against) per 60 minutes.

    Args:
        goalie_stats_df (pd.DataFrame): DataFrame containing goalie statistics with 'xg_against' and 'toi' columns.

    Returns:
        float: The league average xg_against per 60 minutes.

    Raises:
        KeyError: If required columns are missing from the DataFrame.
        ValueError: If no valid goalies with non-zero 'toi' are found.
    """
    # Ensure required columns are present
    required_columns = {'xg_against', 'toi'}
    missing_columns = required_columns - set(goalie_stats_df.columns)
    if missing_columns:
        raise KeyError(f"Missing columns in goalie_stats_df: {', '.join(missing_columns)}")

    # Drop rows with missing or zero 'toi' to avoid division errors
    valid_goalies = goalie_stats_df.dropna(subset=['xg_against', 'toi'])
    valid_goalies = valid_goalies[valid_goalies['toi'] > 0]

    if valid_goalies.empty:
        raise ValueError("No valid goalies with non-zero 'toi' found in goalie_stats_df.")

    # Calculate xg against per 60 minutes for each goalie
    valid_goalies['xg_against_per_60'] = (valid_goalies['xg_against'] / valid_goalies['toi']) * 60

    # Calculate the league average
    league_avg_xg_against_per_60 = valid_goalies['xg_against_per_60'].mean()

    return league_avg_xg_against_per_60

In [50]:
def calculate_xg_against_adj_percentage(lineup_goalie_stats, goalie_avg_xg_against_per_60):
    """
    Calculate the expected goals against adjusted percentage.
    """
    lineup_goalie_stats['adj%'] = (lineup_goalie_stats['xg_against/60'] / goalie_avg_xg_against_per_60) * 100
    return lineup_goalie_stats

In [51]:
def predict_lineup_xgoals(input_date: str, team: str, model, poly, last_n: int = None, player_stats_df: pd.DataFrame=None, goalie_stats_df: pd.DataFrame=None) -> pd.DataFrame:
    """
    Predicts expected goals (xgoals) for a team's lineup using player metrics and regression model.
    Processes the team's lineup for a given date and predicts GPM for each player.
    This function performs the following steps:
        1. Calls `nst_on_ice_scraper` for the input date minus one day.
        2. Extracts the team's lineup using `extract_team_lineup`.
        3. Retrieves skater statistics with `get_skater_stats`.
        4. Calculates `min%` using `calculate_min_percentage`.
        5. Computes `adj_min` using `calculate_adj_min`.
        6. Determines `ixg_per_60` using `calculate_ixg_per_60`.
        7. Predicts `gpm` using the loaded polynomial regression model.
        8. Calculcates x_goals by multiplying gpm by adj_min    
    Args:
        input_date (str): The reference date in 'YYYY-MM-DD' format.
        team (str): The three-letter team code (e.g., 'TOR').
        model: Loaded regression model.
        poly: Loaded PolynomialFeatures transformer.
        
    Returns:
        pd.DataFrame: Updated DataFrame with calculated metrics and predicted GPM.
    
    Raises:
        ValueError: If any step in the data processing pipeline fails.
    """

    try:
        # Step 1: Calculate the date minus one day
        reference_datetime = datetime.strptime(input_date, '%Y-%m-%d') - timedelta(days=1)
        reference_date_str = reference_datetime.strftime('%Y-%m-%d')
        print(f"Fetching data for reference date: {reference_date_str}")

        # Step 1b: If player_stats_df is not provided, fetch them
        if player_stats_df is None or player_stats_df.empty:
            player_stats_df = nst_on_ice_scraper(
                startdate='',
                enddate=reference_date_str,
                last_n=last_n,
                rate='y'
            )
            print("Player and goalie statistics fetched successfully.")
        else:
            print("Player and goalie statistics already provided.")

        if goalie_stats_df is None or goalie_stats_df.empty:
            goalie_stats_df = nst_on_ice_scraper(
                startdate='',
                enddate=reference_date_str,
                pos='G',
                rate='y',
                stdoi='g',
                last_n=last_n
            )
            print("Goalie statistics fetched successfully.")
        else:
            print("Goalie statistics already provided.")

        # Step 3: Extract team lineup for the input date and team
        lineup = extract_team_lineup(team, input_date)
        print(f"Lineup extracted for team {team} on {input_date}.")

        # Step 4: Get skater statistics for the lineup
        lineup_skater_stats = get_skater_stats(lineup, player_stats_df)

        # Step 5: Calculate min%
        lineup_skater_stats = calculate_min_percentage(lineup_skater_stats)

        # Step 6: Calculate adj_min
        total_min_percentage = sum_min_percentage(lineup_skater_stats)
        calculate_adj_min(lineup_skater_stats, total_min_percentage)

        # Step 7: Predict gpm using the polynomial regression model
        lineup_skater_stats = add_gpm_to_lineup(lineup_skater_stats, model, poly)

        # Step 8: Calculate x_goals
        lineup_skater_stats = calculate_x_goals(lineup_skater_stats)

        # Step 9: Get goalie stats for the lineup
        lineup_goalie_stats = get_goalie_stats(lineup, goalie_stats_df)
        
        # Step 10: Calculate league average xg_against_per_60
        goalie_avg_xg_against_per_60 = goalie_stats_df['xg_against/60'].mean()

        lineup_goalie_stats = calculate_xg_against_adj_percentage(lineup_goalie_stats, goalie_avg_xg_against_per_60)
        
        return lineup_skater_stats, lineup_goalie_stats

    except Exception as e:
        print(f"An error occurred during processing: {e}")
        raise

In [52]:
def load_models(model_filepath='../models/polynomial_model_degree_1.pkl',
               poly_filepath='../models/polynomial_features_degree_1.pkl'):
    """
    Loads the regression model and PolynomialFeatures transformer from the specified file paths.

    Args:
        model_filepath (str): Path to the saved regression model pickle file.
        poly_filepath (str): Path to the saved PolynomialFeatures transformer pickle file.

    Returns:
        tuple: A tuple containing the loaded regression model and PolynomialFeatures transformer.

    Raises:
        FileNotFoundError: If either of the specified files does not exist.
        pickle.UnpicklingError: If there is an error unpickling the files.
    """
    try:
        # Load the regression model
        with open(model_filepath, 'rb') as model_file:
            loaded_model = pickle.load(model_file)
        print(f"Model loaded from {model_filepath}")

        # Load the PolynomialFeatures transformer
        with open(poly_filepath, 'rb') as poly_file:
            loaded_poly = pickle.load(poly_file)
        print(f"PolynomialFeatures transformer loaded from {poly_filepath}")

        return loaded_model, loaded_poly

    except FileNotFoundError as fnf_error:
        print(f"Error: {fnf_error}")
        raise
    except pickle.UnpicklingError as pickle_error:
        print(f"Error loading pickle files: {pickle_error}")
        raise
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        raise

# Example usage
# Load the models using the new function
# loaded_model, loaded_poly = load_models()

# # Call the function with desired date and team
# input_date = '2024-12-02'
# team = 'TOR'
# lineup_skater_stats, lineup_goalie_stats = predict_lineup_xgoals(input_date, team, loaded_model, loaded_poly)

In [53]:
def calculate_predicted_goals(matchups_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate predicted goals by multiplying x_goals with opposing team's goalie adjustment percentage.
    
    Args:
        matchups_df: DataFrame containing x_goals and adj% columns from process_matchups_for_date
        
    Returns:
        DataFrame with added predicted goals columns
    """
    # Convert percentages to multipliers (e.g. 105% -> 1.05)
    matchups_df = matchups_df.copy()
    matchups_df['away_adj%'] = matchups_df['away_adj%'] / 100
    matchups_df['home_adj%'] = matchups_df['home_adj%'] / 100
    
    # Calculate predicted goals
    matchups_df['away_pred_goals'] = matchups_df['away_x_goals'] * matchups_df['home_adj%']
    matchups_df['home_pred_goals'] = matchups_df['home_x_goals'] * matchups_df['away_adj%']
    
    return matchups_df

In [54]:
def calculate_win_probabilities(matchups_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate home and away win probabilities based on predicted goals.
    
    Args:
        matchups_df (pd.DataFrame): DataFrame containing 'home_pred_goals' and 'away_pred_goals'.
    
    Returns:
        pd.DataFrame: DataFrame with added 'home_win_prob' and 'away_win_prob' columns, and selected columns.
    """
    HIA = 0.28 # home ice advantage
    HOME = matchups_df['home_pred_goals']
    AWAY = matchups_df['away_pred_goals']
    
    matchups_df['home_win_prob'] = 1 / (1 + np.exp((-(HIA + HOME - AWAY) / 1.318)))
    matchups_df['away_win_prob'] = 1 - matchups_df['home_win_prob']
    
    return matchups_df[[
        'date', 'game_id', 'away_team', 'home_team', 
        'away_x_goals', 'away_adj%', 'home_x_goals', 'home_adj%', 
        'away_pred_goals', 'home_pred_goals',
        'away_win_prob', 'home_win_prob'
    ]]

In [55]:
def process_matchups_for_date(
    input_date: str,
    model,
    poly,
    sportsbook: Optional[str] = None,
    last_n: int = None
) -> pd.DataFrame:
    """
    Processes all matchup games for a given date by extracting team lineups, their statistics,
    predicting expected goals, and optionally retrieving team moneyline odds from a specified sportsbook.
    
    Args:
        input_date (str): The reference date in 'YYYY-MM-DD' format.
        model: Loaded regression model.
        poly: Loaded PolynomialFeatures transformer.
        sportsbook (str, optional): The name of the sportsbook to retrieve moneyline odds from.
        last_n (int, optional): Number of last games to consider for stats. Defaults to None.
    
    Returns:
        pd.DataFrame: A DataFrame with calculated metrics, predicted goals, win probabilities,
                      and optionally moneyline odds.
    """
    try:
        # Step 1: Calculate the date minus one day
        reference_datetime = datetime.strptime(input_date, '%Y-%m-%d') - timedelta(days=1)
        reference_date_str = reference_datetime.strftime('%Y-%m-%d')
        print(f"Fetching data for reference date: {reference_date_str}")

        # Step 2: Call nst_on_ice_scraper for player and goalie stats
        player_stats_df = nst_on_ice_scraper(
            startdate='',
            enddate=reference_date_str,
            rate='y',
            last_n=last_n
        )
        goalie_stats_df = nst_on_ice_scraper(
            startdate='',
            enddate=reference_date_str,
            pos='G',
            rate='y',
            stdoi='g',
            last_n=last_n
        )
        print("Player and goalie statistics fetched successfully.")

        # Step 3: Get matchup games
        temp_data = get_matchup_games(input_date, input_date)
        game_ids = temp_data.get('game_ids', {}).get('id', [])
        game_dates = temp_data.get('game_ids', {}).get('date', [])

        if not game_ids:
            print(f"No games found for the date {input_date}.")
            return pd.DataFrame()

        results = []

        # Step 4: Process each game
        for game_id, game_date in zip(game_ids, game_dates):
            print(f"\nProcessing Game ID: {game_id} on Date: {game_date}")

            # Get the cleaned boxscore
            boxscore = get_game_boxscore(game_id, clean=True)
            away_team = boxscore.get('away_team')
            home_team = boxscore.get('home_team')

            if not away_team or not home_team:
                print(f"Could not extract teams for Game ID: {game_id}. Skipping.")
                continue

            print(f"Away Team: {away_team}, Home Team: {home_team}")

            # Process lineups and calculate stats for both teams
            away_skater_stats, away_goalie_stats = predict_lineup_xgoals(
                game_date, away_team, model, poly, last_n=last_n, player_stats_df=player_stats_df, goalie_stats_df=goalie_stats_df
            )
            home_skater_stats, home_goalie_stats = predict_lineup_xgoals(
                game_date, home_team, model, poly, last_n=last_n, player_stats_df=player_stats_df, goalie_stats_df=goalie_stats_df
            )

            # Calculate team metrics
            away_x_goals = away_skater_stats['x_goals'].sum()
            away_adj_percentage = away_goalie_stats.iloc[0]['adj%'] if not away_goalie_stats.empty else np.nan
            home_x_goals = home_skater_stats['x_goals'].sum()
            home_adj_percentage = home_goalie_stats.iloc[0]['adj%'] if not home_goalie_stats.empty else np.nan

            # Get moneyline odds if sportsbook is specified
            away_odds = np.nan
            home_odds = np.nan
            if sportsbook:
                away_odds_list = get_team_moneyline_odds(away_team, game_date, sportsbook)
                if away_odds_list:
                    away_odds = away_odds_list[0]['price']
                
                home_odds_list = get_team_moneyline_odds(home_team, game_date, sportsbook)
                if home_odds_list:
                    home_odds = home_odds_list[0]['price']

            # Store results
            results.append({
                'date': game_date,
                'game_id': game_id,
                'away_team': away_team,
                'home_team': home_team,
                'away_x_goals': away_x_goals,
                'away_adj%': away_adj_percentage,
                'home_x_goals': home_x_goals,
                'home_adj%': home_adj_percentage,
                'away_moneyline_odds': away_odds,
                'home_moneyline_odds': home_odds
            })

        # Convert results to DataFrame
        results_df = pd.DataFrame(results)
        
        if not results_df.empty:
            # Store moneyline odds before calculations if they exist
            moneyline_cols = ['away_moneyline_odds', 'home_moneyline_odds']
            odds_data = None
            if sportsbook and all(col in results_df.columns for col in moneyline_cols):
                odds_data = results_df[moneyline_cols].copy()

            # Calculate predicted goals and win probabilities
            results_df = calculate_predicted_goals(results_df)
            results_df = calculate_win_probabilities(results_df)
            
            # Restore moneyline odds if they existed
            if odds_data is not None:
                results_df[moneyline_cols] = odds_data
            
            # Define columns based on whether sportsbook was provided
            columns = [
                'date', 'game_id', 'away_team', 'home_team',
                'away_x_goals', 'away_adj%', 'home_x_goals', 'home_adj%',
                'away_pred_goals', 'home_pred_goals',
                'away_win_prob', 'home_win_prob'
            ]
            
            # Only include moneyline columns if sportsbook was provided and odds exist
            if odds_data is not None:
                columns.extend(moneyline_cols)
            
            # Reorder columns
            results_df = results_df[columns]

        print("\nAll matchups processed successfully.")
        return results_df

    except Exception as e:
        print(f"An error occurred during processing: {e}")
        raise

In [56]:
# matchups = process_matchups_for_date('2024-12-08', loaded_model, loaded_poly, last_n=15)

In [57]:
# Load the models using the existing load_models function
loaded_model, loaded_poly = load_models()

# Define the input parameters
input_date = '2024-12-08'
sportsbook = 'fanduel'  # Optional: specify the sportsbook name

# Call the updated process_matchups_for_date function with sportsbook
matchups = process_matchups_for_date(
    input_date=input_date,
    model=loaded_model,
    poly=loaded_poly,
    sportsbook=sportsbook,  # Provide the sportsbook name or omit if not needed
    last_n=15
)

# Display the resulting DataFrame
matchups

Model loaded from ../models/polynomial_model_degree_1.pkl
PolynomialFeatures transformer loaded from ../models/polynomial_features_degree_1.pkl
Fetching data for reference date: 2024-12-07
Player and goalie statistics fetched successfully.

Processing Game ID: 2024020436 on Date: 2024-12-08
Away Team: SEA, Home Team: NYR
Fetching data for reference date: 2024-12-07
Player and goalie statistics already provided.
Goalie statistics already provided.
Game ID: 2024020420
Added player 'Chandler Stephenson' to Forwards slot 1.
Added player 'Matty Beniers' to Forwards slot 2.
Added player 'Tye Kartye' to Forwards slot 3.
Added player 'Brandon Tanev' to Forwards slot 4.
Added player 'Jaden Schwartz' to Forwards slot 5.
Added player 'Jared McCann' to Forwards slot 6.
Added player 'Eeli Tolvanen' to Forwards slot 7.
Added player 'Oliver Bjorkstrand' to Forwards slot 8.
Added player 'Ryan Winterton' to Forwards slot 9.
Added player 'Shane Wright' to Forwards slot 10.
Added player 'Mitchell Stephen

INFO:root:Retrieving moneyline odds for team: SEA, date: 2024-12-08, sportsbook: fanduel
INFO:root:Retrieving NHL events from DB for date: 2024-12-08
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres


Added player 'Victor Mancini' to Defense slot 6.
Added player 'Igor Shesterkin' to Goalies slot 1.
Added player 'Jonathan Quick' to Goalies slot 2.
Lineup extracted for team NYR on 2024-12-08.


INFO:src.db.base_utils:Successfully established database connection
INFO:root:Retrieved 7 events from the database for date 2024-12-08.
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Completed retrieving moneyline odds.
INFO:root:Retrieving moneyline odds for team: NYR, date: 2024-12-08, sportsbook: fanduel
INFO:root:Retrieving NHL events from DB for date: 2024-12-08
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Retrieved 7 events from the database for date 2024-12-08.
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Completed retrieving moneyline odds.



Processing Game ID: 2024020437 on Date: 2024-12-08
Away Team: TBL, Home Team: VAN
Fetching data for reference date: 2024-12-07
Player and goalie statistics already provided.
Goalie statistics already provided.
Game ID: 2024020413
Added player 'Luke Glendening' to Forwards slot 1.
Added player 'Cam Atkinson' to Forwards slot 2.
Added player 'Conor Geekie' to Forwards slot 3.
Added player 'Nick Paul' to Forwards slot 4.
Added player 'Brayden Point' to Forwards slot 5.
Added player 'Michael Eyssimont' to Forwards slot 6.
Added player 'Zemgus Girgensons' to Forwards slot 7.
Added player 'Brandon Hagel' to Forwards slot 8.
Added player 'Jake Guentzel' to Forwards slot 9.
Added player 'Anthony Cirelli' to Forwards slot 10.
Added player 'Gage Goncalves' to Forwards slot 11.
Added player 'Ryan McDonagh' to Defense slot 1.
Added player 'Darren Raddysh' to Defense slot 2.
Added player 'Nick Perbix' to Defense slot 3.
Added player 'Victor Hedman' to Defense slot 4.
Added player 'Emil Lilleberg' 

...

INFO:root:Retrieving moneyline odds for team: CGY, date: 2024-12-08, sportsbook: fanduel
INFO:root:Retrieving NHL events from DB for date: 2024-12-08
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Retrieved 7 events from the database for date 2024-12-08.
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Completed retrieving moneyline odds.
INFO:root:Retrieving moneyline odds for team: DAL, date: 2024-12-08, sportsbook: fanduel
INFO:root:Retrieving NHL events from DB for date: 2024-12-08
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Retrieved 7 events from the databas

Added player 'Casey DeSmith' to Goalies slot 1.
Added player 'Jake Oettinger' to Goalies slot 2.
Lineup extracted for team DAL on 2024-12-08.


INFO:src.db.base_utils:Successfully established database connection
INFO:root:Completed retrieving moneyline odds.



All matchups processed successfully.


,date,game_id,away_team,home_team,away_x_goals,away_adj%,home_x_goals,home_adj%,away_pred_goals,home_pred_goals,away_win_prob,home_win_prob,away_moneyline_odds,home_moneyline_odds
0,2024-12-08,2024020436,SEA,NYR,2.258271,0.904408,2.220763,1.328599,3.000336,2.008475,0.631837,0.368163,142,-172
1,2024-12-08,2024020437,TBL,VAN,2.688003,0.908409,1.624222,0.836377,2.248184,1.475459,0.592387,0.407613,-126,105
2,2024-12-08,2024020438,NYI,OTT,2.147845,0.680307,2.293332,0.828373,1.779218,1.560169,0.488441,0.511559,118,-142
3,2024-12-08,2024020439,CBJ,WPG,2.073545,1.560703,2.115903,0.976440,2.024693,3.302297,0.234730,0.765270,126,-152
4,2024-12-08,2024020440,COL,NJD,2.729057,0.968436,3.504225,0.744335,2.031334,3.393619,0.223386,0.776614,130,-156
5,2024-12-08,2024020441,UTA,PHI,2.720273,1.220550,2.906995,0.772348,2.100998,3.548133,0.212416,0.787584,-105,-114
6,2024-12-08,2024020442,CGY,DAL,2.657023,0.828373,2.402156,1.088491,2.892145,1.989882,0.615887,0.384113,188,-230


In [58]:
def process_matchups_for_date_range(
    start_date: str,
    end_date: str,
    model,
    poly,
    sportsbook: Optional[str] = None,
    last_n: int = None
) -> pd.DataFrame:
    """
    Processes matchup data for each date within the specified range.

    Args:
        start_date (str): The start date in 'YYYY-MM-DD' format.
        end_date (str): The end date in 'YYYY-MM-DD' format.
        model: Loaded regression model.
        poly: Loaded PolynomialFeatures transformer.
        sportsbook (str, optional): The name of the sportsbook to retrieve moneyline odds from.
        last_n (int, optional): Number of last games to consider for stats. Defaults to None.

    Returns:
        pd.DataFrame: Aggregated DataFrame containing matchup data for all dates in the range.

    Raises:
        ValueError: If the date format is incorrect or if start_date is after end_date.
    """
    from datetime import datetime, timedelta

    # Validate date formats
    try:
        start_dt = datetime.strptime(start_date, '%Y-%m-%d')
        end_dt = datetime.strptime(end_date, '%Y-%m-%d')
    except ValueError as ve:
        raise ValueError("start_date and end_date must be in 'YYYY-MM-DD' format.") from ve

    if start_dt > end_dt:
        raise ValueError("start_date must be earlier than or equal to end_date.")

    # Initialize list to hold DataFrames for each date
    all_matchups = []

    current_date = start_dt
    while current_date <= end_dt:
        date_str = current_date.strftime('%Y-%m-%d')
        print(f"\nProcessing matchups for date: {date_str}")

        try:
            daily_matchups = process_matchups_for_date(
                input_date=date_str,
                model=model,
                poly=poly,
                sportsbook=sportsbook,
                last_n=last_n
            )

            if not daily_matchups.empty:
                all_matchups.append(daily_matchups)
                print(f"Successfully processed matchups for {date_str}.")
            else:
                print(f"No matchups found for {date_str}.")

        except Exception as e:
            print(f"Failed to process matchups for {date_str}: {e}")

        # Move to the next day
        current_date += timedelta(days=1)

    # Concatenate all DataFrames
    if all_matchups:
        aggregated_df = pd.concat(all_matchups, ignore_index=True)
        print("\nAll matchups processed successfully.")
        return aggregated_df
    else:
        print("No matchups were processed.")
        return pd.DataFrame()

In [59]:
# Define the date range
start_date = '2024-11-01'
end_date = '2024-12-31'

# Specify the sportsbook if needed
sportsbook = 'fanduel'  # Change to desired sportsbook or set to None

# Call the function with the date range
aggregated_matchups = process_matchups_for_date_range(
    start_date=start_date,
    end_date=end_date,
    model=loaded_model,
    poly=loaded_poly,
    sportsbook=sportsbook,
    last_n=15
)

# Display the aggregated DataFrame
aggregated_matchups


Processing matchups for date: 2024-11-01
Fetching data for reference date: 2024-10-31
Player and goalie statistics fetched successfully.

Processing Game ID: 2024020168 on Date: 2024-11-01
Away Team: FLA, Home Team: DAL
Fetching data for reference date: 2024-10-31
Player and goalie statistics already provided.
Goalie statistics already provided.
Game ID: 2024020140
Added player 'Sam Bennett' to Forwards slot 1.
Added player 'A.J. Greer' to Forwards slot 2.
Added player 'Sam Reinhart' to Forwards slot 3.
Added player 'Anton Lundell' to Forwards slot 4.
Added player 'Aleksander Barkov' to Forwards slot 5.
Added player 'Evan Rodrigues' to Forwards slot 6.
Added player 'Matthew Tkachuk' to Forwards slot 7.
Added player 'Carter Verhaeghe' to Forwards slot 8.
Added player 'Mackie Samoskevich' to Forwards slot 9.
Added player 'Eetu Luostarinen' to Forwards slot 10.
Added player 'Patrick Giles' to Forwards slot 11.
Added player 'Jesper Boqvist' to Forwards slot 12.
Added player 'Aaron Ekblad'

INFO:root:Retrieving moneyline odds for team: DAL, date: 2024-11-02, sportsbook: fanduel
INFO:root:Retrieving NHL events from DB for date: 2024-11-02
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Retrieved 11 events from the database for date 2024-11-02.
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection


Added player 'Gustav Forsling' to Defense slot 4.
Added player 'Niko Mikkola' to Defense slot 5.
Added player 'Nate Schmidt' to Defense slot 6.
Added player 'Spencer Knight' to Goalies slot 1.
Added player 'Sergei Bobrovsky' to Goalies slot 2.
Lineup extracted for team FLA on 2024-11-02.


INFO:root:Completed retrieving moneyline odds.
INFO:root:Retrieving moneyline odds for team: FLA, date: 2024-11-02, sportsbook: fanduel
INFO:root:Retrieving NHL events from DB for date: 2024-11-02
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Retrieved 11 events from the database for date 2024-11-02.
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Completed retrieving moneyline odds.



Processing Game ID: 2024020175 on Date: 2024-11-02
Away Team: BOS, Home Team: PHI
Fetching data for reference date: 2024-11-01
Player and goalie statistics already provided.
Goalie statistics already provided.
Game ID: 2024020165
Added player 'Trent Frederic' to Forwards slot 1.
Added player 'Charlie Coyle' to Forwards slot 2.
Added player 'Pavel Zacha' to Forwards slot 3.
Added player 'John Beecher' to Forwards slot 4.
Added player 'Elias Lindholm' to Forwards slot 5.
Added player 'Cole Koepke' to Forwards slot 6.
Added player 'Mark Kastelic' to Forwards slot 7.
Added player 'Max Jones' to Forwards slot 8.
Added player 'Matthew Poitras' to Forwards slot 9.
Added player 'Justin Brazeau' to Forwards slot 10.
Added player 'Brad Marchand' to Forwards slot 11.
Added player 'David Pastrnak' to Forwards slot 12.
Added player 'Mason Lohrei' to Defense slot 1.
Added player 'Brandon Carlo' to Defense slot 2.
Added player 'Hampus Lindholm' to Defense slot 3.
Added player 'Andrew Peeke' to Defen

...

INFO:root:Retrieving moneyline odds for team: UTA, date: 2024-12-31, sportsbook: fanduel
INFO:root:Retrieving NHL events from DB for date: 2024-12-31
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Retrieved 13 events from the database for date 2024-12-31.
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres


Added player 'Troy Stecher' to Defense slot 6.
Added player 'Calvin Pickard' to Goalies slot 1.
Added player 'Stuart Skinner' to Goalies slot 2.
Lineup extracted for team EDM on 2024-12-31.


INFO:src.db.base_utils:Successfully established database connection
INFO:root:Completed retrieving moneyline odds.
INFO:root:Retrieving moneyline odds for team: EDM, date: 2024-12-31, sportsbook: fanduel
INFO:root:Retrieving NHL events from DB for date: 2024-12-31
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Retrieved 13 events from the database for date 2024-12-31.
INFO:src.db.base_utils:Attempting connection with: host=localhost, port=5432, dbname=the_odds_db, user=postgres
INFO:src.db.base_utils:Successfully established database connection
INFO:root:Completed retrieving moneyline odds.



All matchups processed successfully.
Successfully processed matchups for 2024-12-31.

All matchups processed successfully.


,date,game_id,away_team,home_team,away_x_goals,away_adj%,home_x_goals,home_adj%,away_pred_goals,home_pred_goals,away_win_prob,home_win_prob,away_moneyline_odds,home_moneyline_odds
0,2024-11-02,2024020174,DAL,FLA,2.832456,0.980179,2.827360,0.952513,2.697951,2.771320,0.433371,0.566629,-110.0,-110.0
1,2024-11-02,2024020175,BOS,PHI,2.083253,0.964370,2.200495,0.822086,1.712613,2.122092,0.372122,0.627878,-128.0,106.0
2,2024-11-02,2024020176,CHI,LAK,2.538116,1.075035,3.265839,0.881371,2.237022,3.510893,0.235239,0.764761,155.0,-188.0
3,2024-11-02,2024020177,CBJ,WSH,2.017497,1.082940,3.033081,0.964370,1.945613,3.284645,0.226461,0.773539,220.0,-275.0
4,2024-11-02,2024020178,SEA,OTT,2.099344,1.055274,2.207407,1.134320,2.381328,2.329419,0.456843,0.543157,104.0,-125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,2024-12-31,2024020597,WPG,COL,2.932707,1.442785,2.483553,0.990924,2.906089,3.583232,0.326027,0.673973,172.0,-210.0
375,2024-12-31,2024020598,NJD,ANA,2.940491,0.864085,2.702884,1.050379,3.088630,2.335523,0.588788,0.411212,-285.0,230.0
376,2024-12-31,2024020599,PHI,SJS,2.445574,0.673828,2.030340,1.185145,2.898359,1.368100,0.720834,0.279166,-182.0,150.0
377,2024-12-31,2024020600,VAN,CGY,2.305257,0.903722,2.725536,0.975069,2.247785,2.463128,0.407133,0.592867,112.0,-134.0


In [60]:
aggregated_matchups.head()

,date,game_id,away_team,home_team,away_x_goals,away_adj%,home_x_goals,home_adj%,away_pred_goals,home_pred_goals,away_win_prob,home_win_prob,away_moneyline_odds,home_moneyline_odds
0,2024-11-02,2024020174,DAL,FLA,2.832456,0.980179,2.827360,0.952513,2.697951,2.771320,0.433371,0.566629,-110.0,-110.0
1,2024-11-02,2024020175,BOS,PHI,2.083253,0.964370,2.200495,0.822086,1.712613,2.122092,0.372122,0.627878,-128.0,106.0
2,2024-11-02,2024020176,CHI,LAK,2.538116,1.075035,3.265839,0.881371,2.237022,3.510893,0.235239,0.764761,155.0,-188.0
3,2024-11-02,2024020177,CBJ,WSH,2.017497,1.082940,3.033081,0.964370,1.945613,3.284645,0.226461,0.773539,220.0,-275.0
4,2024-11-02,2024020178,SEA,OTT,2.099344,1.055274,2.207407,1.134320,2.381328,2.329419,0.456843,0.543157,104.0,-125.0


In [35]:
def add_results_to_matchups_df(matchup_df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds game scores and results to the matchup DataFrame by retrieving data from get_game_boxscore.

    Args:
        matchup_df (pd.DataFrame): DataFrame containing at least a 'game_id' column.

    Returns:
        pd.DataFrame: Updated DataFrame with 'away_score', 'home_score', and 'result' columns added.

    Raises:
        KeyError: If 'game_id' column is not present in the DataFrame.
    """
    if 'game_id' not in matchup_df.columns:
        raise KeyError("The DataFrame must contain a 'game_id' column.")

    # Initialize lists to store scores and results
    away_scores = []
    home_scores = []
    results = []

    for index, row in matchup_df.iterrows():
        game_id = row['game_id']
        try:
            boxscore = get_game_boxscore(game_id, clean=False)
            
            # Extract scores
            away_score = boxscore.get('awayTeam', {}).get('score')
            home_score = boxscore.get('homeTeam', {}).get('score')

            if away_score is None or home_score is None:
                logging.warning(f"Score information missing for game ID {game_id}.")
                away_scores.append(None)
                home_scores.append(None)
                results.append(None)
                continue

            away_scores.append(away_score)
            home_scores.append(home_score)

            # Determine result
            if away_score > home_score:
                result = 'a'
            elif home_score > away_score:
                result = 'h'
            else:
                result = 't'  # Note: NHL games typically go to overtime/shootout to avoid ties

            results.append(result)
        
        except Exception as e:
            logging.error(f"Failed to retrieve boxscore for game ID {game_id}: {e}")
            away_scores.append(None)
            home_scores.append(None)
            results.append(None)

    # Add new columns to the DataFrame
    matchup_df['a_score'] = away_scores
    matchup_df['h_score'] = home_scores
    matchup_df['res'] = results

    return matchup_df

In [61]:
# Add results to the DataFrame
aggregated_matchups = add_results_to_matchups_df(aggregated_matchups)

# Display the updated DataFrame
aggregated_matchups

,date,game_id,away_team,home_team,away_x_goals,away_adj%,home_x_goals,home_adj%,away_pred_goals,home_pred_goals,away_win_prob,home_win_prob,away_moneyline_odds,home_moneyline_odds,away_score,home_score,result
0,2024-11-02,2024020174,DAL,FLA,2.832456,0.980179,2.827360,0.952513,2.697951,2.771320,0.433371,0.566629,-110.0,-110.0,2,4,Home Win
1,2024-11-02,2024020175,BOS,PHI,2.083253,0.964370,2.200495,0.822086,1.712613,2.122092,0.372122,0.627878,-128.0,106.0,3,0,Away Win
2,2024-11-02,2024020176,CHI,LAK,2.538116,1.075035,3.265839,0.881371,2.237022,3.510893,0.235239,0.764761,155.0,-188.0,4,3,Away Win
3,2024-11-02,2024020177,CBJ,WSH,2.017497,1.082940,3.033081,0.964370,1.945613,3.284645,0.226461,0.773539,220.0,-275.0,2,7,Home Win
4,2024-11-02,2024020178,SEA,OTT,2.099344,1.055274,2.207407,1.134320,2.381328,2.329419,0.456843,0.543157,104.0,-125.0,0,3,Home Win
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
374,2024-12-31,2024020597,WPG,COL,2.932707,1.442785,2.483553,0.990924,2.906089,3.583232,0.326027,0.673973,172.0,-210.0,2,5,Home Win
375,2024-12-31,2024020598,NJD,ANA,2.940491,0.864085,2.702884,1.050379,3.088630,2.335523,0.588788,0.411212,-285.0,230.0,2,3,Home Win
376,2024-12-31,2024020599,PHI,SJS,2.445574,0.673828,2.030340,1.185145,2.898359,1.368100,0.720834,0.279166,-182.0,150.0,4,0,Away Win
377,2024-12-31,2024020600,VAN,CGY,2.305257,0.903722,2.725536,0.975069,2.247785,2.463128,0.407133,0.592867,112.0,-134.0,1,3,Home Win


In [62]:
aggregated_matchups.to_csv('ash_01_ml_11-01-24_12-31-24.csv', index=False)